In [14]:
import pandas as pd
import numpy as np

from scipy import stats
import math

from mplsoccer import PyPizza, add_image, FontManager
import matplotlib.pyplot as plt

## Data Manipulation

In [15]:
#import csv of premier league defensive stats from fbref. I have uploaded the data but I changed the column names in the csv prior.
df = pd.read_csv(r"C:\\Users\josep\Downloads\Prem PLayers - Sheet1.csv")

In [16]:
df

,Rk,Player,Nation,Pos,Squad,Age,Born,90s,Tackles,Tkls Won,...,Dribbled Past,Successful Pressures,Def 3rd Press,Mid 3rd Press,Att 3rd Press,Blocks,Interceptions,Tkl+Int,Clearances,Errors
0,1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,30-259,1990,19.7,1.62,0.96,...,1.37,3.45,6.55,2.84,1.02,1.57,1.73,3.35,2.03,0.05
1,2,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,23-225,1997,11.6,0.69,0.34,...,0.43,3.28,1.12,6.12,6.72,0.78,0.17,0.86,1.47,0.00
2,3,Che Adams\Che-Adams,sco SCO,FW,Southampton,24-306,1996,28.1,1.03,0.57,...,0.57,4.70,1.71,5.41,7.12,0.89,0.36,1.39,0.57,0.04
3,4,Tosin Adarabioyo\Tosin-Adarabioyo,eng ENG,DF,Fulham,23-233,1997,29.8,1.28,0.77,...,0.54,1.78,3.66,1.64,0.23,1.17,1.68,2.95,7.38,0.03
4,5,Adrián\Adrian,es ESP,GK,Liverpool,34-132,1987,3.0,0.00,0.00,...,0.00,0.00,0.33,0.00,0.00,0.00,0.00,0.00,0.00,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,517,Andi Zeqiri\Andi-Zeqiri,ch SUI,DFFW,Brighton,21-327,1999,1.5,2.00,2.00,...,0.67,6.00,5.33,8.67,6.67,1.33,0.00,2.00,0.67,0.00
517,518,Oleksandr Zinchenko\Oleksandr-Zinchenko,ua UKR,DF,Manchester City,24-151,1996,14.4,1.46,0.90,...,0.97,2.99,3.61,3.96,1.94,1.46,1.32,2.78,2.85,0.00
518,519,Hakim Ziyech\Hakim-Ziyech,ma MAR,FWMF,Chelsea,28-057,1993,12.7,0.79,0.55,...,1.18,4.80,3.70,7.09,6.61,1.10,0.71,1.50,0.08,0.00
519,520,Kurt Zouma\Kurt-Zouma,fr FRA,DF,Chelsea,26-200,1994,22.5,0.93,0.58,...,0.22,1.78,2.31,2.36,0.18,1.02,1.16,2.09,5.69,0.09


You can see from the table that the imported data set from FBref has a weird way of displaying the footballers name. The following code gets rid of the text that comes after the \

In [17]:
df['Player'] = df['Player'].str.split('\\',expand=True)[0]
# displays the player names in a nicer way

In [18]:
df

,Rk,Player,Nation,Pos,Squad,Age,Born,90s,Tackles,Tkls Won,...,Dribbled Past,Successful Pressures,Def 3rd Press,Mid 3rd Press,Att 3rd Press,Blocks,Interceptions,Tkl+Int,Clearances,Errors
0,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,30-259,1990,19.7,1.62,0.96,...,1.37,3.45,6.55,2.84,1.02,1.57,1.73,3.35,2.03,0.05
1,2,Tammy Abraham,eng ENG,FW,Chelsea,23-225,1997,11.6,0.69,0.34,...,0.43,3.28,1.12,6.12,6.72,0.78,0.17,0.86,1.47,0.00
2,3,Che Adams,sco SCO,FW,Southampton,24-306,1996,28.1,1.03,0.57,...,0.57,4.70,1.71,5.41,7.12,0.89,0.36,1.39,0.57,0.04
3,4,Tosin Adarabioyo,eng ENG,DF,Fulham,23-233,1997,29.8,1.28,0.77,...,0.54,1.78,3.66,1.64,0.23,1.17,1.68,2.95,7.38,0.03
4,5,Adrián,es ESP,GK,Liverpool,34-132,1987,3.0,0.00,0.00,...,0.00,0.00,0.33,0.00,0.00,0.00,0.00,0.00,0.00,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,517,Andi Zeqiri,ch SUI,DFFW,Brighton,21-327,1999,1.5,2.00,2.00,...,0.67,6.00,5.33,8.67,6.67,1.33,0.00,2.00,0.67,0.00
517,518,Oleksandr Zinchenko,ua UKR,DF,Manchester City,24-151,1996,14.4,1.46,0.90,...,0.97,2.99,3.61,3.96,1.94,1.46,1.32,2.78,2.85,0.00
518,519,Hakim Ziyech,ma MAR,FWMF,Chelsea,28-057,1993,12.7,0.79,0.55,...,1.18,4.80,3.70,7.09,6.61,1.10,0.71,1.50,0.08,0.00
519,520,Kurt Zouma,fr FRA,DF,Chelsea,26-200,1994,22.5,0.93,0.58,...,0.22,1.78,2.31,2.36,0.18,1.02,1.16,2.09,5.69,0.09


The player names are now displayed nicely. 

In [19]:
df = df.drop(['Rk','Nation','Pos','Squad','Age','Born'],axis=1).reset_index()
# Drops any columns that we are not interested in to make the pizza plot 

In [20]:
df

,index,Player,90s,Tackles,Tkls Won,Tkls vs Dribbles,Dribbled Past,Successful Pressures,Def 3rd Press,Mid 3rd Press,Att 3rd Press,Blocks,Interceptions,Tkl+Int,Clearances,Errors
0,0,Patrick van Aanholt,19.7,1.62,0.96,0.71,1.37,3.45,6.55,2.84,1.02,1.57,1.73,3.35,2.03,0.05
1,1,Tammy Abraham,11.6,0.69,0.34,0.09,0.43,3.28,1.12,6.12,6.72,0.78,0.17,0.86,1.47,0.00
2,2,Che Adams,28.1,1.03,0.57,0.21,0.57,4.70,1.71,5.41,7.12,0.89,0.36,1.39,0.57,0.04
3,3,Tosin Adarabioyo,29.8,1.28,0.77,0.57,0.54,1.78,3.66,1.64,0.23,1.17,1.68,2.95,7.38,0.03
4,4,Adrián,3.0,0.00,0.00,0.00,0.00,0.00,0.33,0.00,0.00,0.00,0.00,0.00,0.00,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,516,Andi Zeqiri,1.5,2.00,2.00,0.00,0.67,6.00,5.33,8.67,6.67,1.33,0.00,2.00,0.67,0.00
517,517,Oleksandr Zinchenko,14.4,1.46,0.90,0.97,0.97,2.99,3.61,3.96,1.94,1.46,1.32,2.78,2.85,0.00
518,518,Hakim Ziyech,12.7,0.79,0.55,0.39,1.18,4.80,3.70,7.09,6.61,1.10,0.71,1.50,0.08,0.00
519,519,Kurt Zouma,22.5,0.93,0.58,0.27,0.22,1.78,2.31,2.36,0.18,1.02,1.16,2.09,5.69,0.09


We only want to compare Joao Cancelo to players who have played more than 12 full matches. This limits the data to only players who play regularly for their clubs. Thus, 